<a href="https://colab.research.google.com/github/josefmadero/hello-world/blob/master/nocountry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
from google.colab import userdata

drive.mount('/content/drive')


# Reemplaza 'your_token_here' con tu token de acceso personal de GitHub

#Para clonar un repositorio privado de GitHub en Google Colab, sigue estos pasos:

#Obtener un token de acceso personal (PAT):

#Ve a tu perfil de GitHub y selecciona Settings.
#En el menú lateral, ve a Developer settings > Personal access tokens > Tokens (classic).
#Crea un nuevo token con permisos para repo (esto te permite clonar repositorios privados).
#Guarda el token generado, ya que lo necesitarás.
# Reemplaza 'No-Country-simulation/equipo-c23-17-data' con la URL del repositorio
!git clone https://{userdata.get('github_pat')}@github.com/No-Country-simulation/equipo-c23-17-data.git



Mounted at /content/drive
Cloning into 'equipo-c23-17-data'...
remote: Enumerating objects: 168, done.
remote: Total 168 (delta 0), reused 0 (delta 0), pack-reused 168 (from 1)
Receiving objects: 100% (168/168), 96.61 MiB | 22.21 MiB/s, done.
Resolving deltas: 100% (140/140), done.


Los clientes de Beta Bank se están yendo, cada mes, poco a poco. Los banqueros descubrieron que es más barato salvar a los clientes existentes que atraer nuevos.

Necesitamos predecir si un cliente dejará el banco pronto. Tú tienes los datos sobre el comportamiento pasado de los clientes y la terminación de contratos con el banco.

Crea un modelo con el máximo valor F1 posible. Para aprobar la revisión, necesitas un valor F1 de al menos 0.59. Verifica F1 para el conjunto de prueba.

Además, debes medir la métrica AUC-ROC y compararla con el valor F1.

# **Descarga y prepara los datos.**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.metrics import f1_score, roc_auc_score

In [ ]:
data = pd.read_csv('/content/equipo-c23-17-data/data/churn.csv')

display(data)
print(data.info())

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB
None


In [ ]:
print(data['Tenure'].isnull().sum())

909


In [ ]:
data['Tenure'] = data['Tenure'].fillna(0)
print(data['Tenure'].isnull().sum())

0


Se importaron las librerías necesarias y se analizaron los datos entregados Solo una de las columnas tenía valores ausentes, la columna Tenure con 909 valores NaN, los cuales se reemplazaron por ceros. El resto de los datos no tenían valores ausentes y su tipo de datos correspondía con los valores de cada columna.

# **Examina el equilibrio de clases. Entrena el modelo sin tener en cuenta el desequilibrio. Describe brevemente tus hallazgos.**

In [ ]:
# Examinar las clases

class_distribution = data['Exited'].value_counts()
print("Distribución de clases:")
print(class_distribution)

Distribución de clases:
Exited
0    7963
1    2037
Name: count, dtype: int64


In [ ]:
# Entrenar el modelo

data = pd.get_dummies(data, columns=['Geography', 'Gender'], drop_first=True)

features = data.drop(columns=['RowNumber', 'CustomerId', 'Surname', 'Exited'], axis = 1)
target = data['Exited']


# División del conjunto de datos en dos: conjunto de prueba (20%) y el resto (conjunto de validación + conjuntos de entrenamiento) (80%)

features_80, features_test, target_80, target_test = train_test_split(features, target, test_size=0.20, random_state=12345)

# División del conjunto de datos restante en conjunto de entrenamiento (60%) y conjunto de validación (20%)

features_train, features_valid, target_train, target_valid = train_test_split(features_80, target_80, test_size=0.25, random_state=12345)

Se eliminan las columnas RowNumber, CustomerId, surname porque no son relevantes para la predicción del abandono de clientes.Y exited porque es el target

In [ ]:
model = RandomForestClassifier(random_state=12345)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)

In [ ]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

predicted_valid = model.predict(features_valid)
f1_score_value = f1_score(target_valid, predicted_valid)


print("Modelo sin corregir desequilibrio de clases:")
print()
print("AUC-ROC Score:", auc_roc)
print("F1 Score: ", f1_score_value)

Modelo sin corregir desequilibrio de clases:

AUC-ROC Score: 0.8470257614219249
F1 Score:  0.5654952076677316


La distribución de las clases en la columna 'Exited' muestra un claro desequilibrio:

Clase 0 (El cliente NO se ha ido): 7963 Clase 1 (El cliente se ha ido): 2037

Se eliminaron las columnas 'RowNumber', 'CustomerId', 'Surname' por no ser relevantes para la predicción y la columna 'Exited' por ser el objetivo.

Las variables categóricas 'Geography' y 'Gender' fueron convertidas a variables dummy, transformando las características categóricas en numéricas (variables dummy) mediante la configuración One-Hot Encoding (OHE).

Los datos fueron divididos en conjunto de prueba (20%) y un conjunto de validación + entrenamiento (80%). El conjunto de validación + entrenamiento fue dividido nuevamente en conjunto de entrenamiento (60% del total) y conjunto de validación (20% del total).

Los valores de AUC-ROC y F1 en el modelo sin corregir fueron de:

AUC-ROC Score: 0.8470 F1 Score: 0.5655

la curva ROC es una línea diagonal que va desde la esquina inferior izquierda hasta la esquina superior derecha. Cuanto más se aleje la curva ROC de esta línea diagonal hacia la esquina superior izquierda, mejor será el modelo, ya que indica una mayor relación TVP-TFP.

El valor F1 es la media armónica de recall y precisión. Un valor F1 cercano a cero mostrará que la predicción de la clase 1 ha fallado.

Estos resultados indican que el modelo tiene una buena capacidad para discriminar entre las clases (AUC-ROC alto), pero el F1 Score relativamente bajo sugiere que el modelo podría estar luchando con el desequilibrio de clases, afectando su precisión y recall en la clase minoritaria (clientes que han abandonado).

# **Mejora la calidad del modelo**

In [ ]:
# corregir el desequilibrio de clases.

# Sobremuestreo
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)

    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345
    )

    return features_upsampled, target_upsampled

In [ ]:
# Submuestreo

def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)]
        + [features_ones]
    )
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)]
        + [target_ones]
    )

    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345
    )

    return features_downsampled, target_downsampled

In [ ]:
# Entrenamiento con diferentes conjuntos de entrenamiento y validación para encontrar el mejor modelo y el mejor conjunto de parámetros.

best_f1 = 0
best_model = None

for n_estimators in [10, 25, 50, 75, 100]:
    for max_depth in range(1,20):

        # Sobremuestreo

        features_upsampled, target_upsampled = upsample(features_train, target_train, 10)
        model_upsampled = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=12345)
        model_upsampled.fit(features_upsampled, target_upsampled)
        predictions_valid_upsampled = model_upsampled.predict(features_valid)
        f1_score_upsampled_value = f1_score(target_valid, predictions_valid_upsampled)

        if f1_score_upsampled_value > best_f1:
            best_f1 = f1_score_upsampled_value
            best_model = model_upsampled
            best_params = {'model': 'RandomForest', 'n_estimators': n_estimators, 'max_depth': max_depth}


        # Submuestreo

        features_downsampled, target_downsampled = downsample(features_train, target_train, 0.1)
        model_downsampled = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=12345)
        model_downsampled.fit(features_downsampled, target_downsampled)
        predictions_valid_downsampled = model_downsampled.predict(features_valid)
        f1_score_downsampled_value = f1_score(target_valid, predictions_valid_downsampled)

        if f1_score_downsampled_value > best_f1:
            best_f1 = f1_score_downsampled_value
            best_model = model_downsampled
            best_params = {'model': 'RandomForest', 'n_estimators': n_estimators, 'max_depth': max_depth}

In [ ]:
print(best_model)
print('n_estimators: ' ,best_params['n_estimators'])
print('max_depth: ', best_params['max_depth'])

RandomForestClassifier(max_depth=18, random_state=12345)
n_estimators:  100
max_depth:  18


Se aplicaron dos técnicas de corrección del desequilibrio de clases:

El sobremuestreo y el submuestreo son técnicas utilizadas para manejar el desequilibrio de clases en problemas de aprendizaje automático.

Sobremuestreo (Upsampling): Aumentando la cantidad de ejemplos de la clase minoritaria (clientes que han abandonado) para igualarla a la clase mayoritaria.
Submuestreo (Downsampling): Reduciendo la cantidad de ejemplos de la clase mayoritaria (clientes que no han abandonado) para igualarla a la clase minoritaria.
Se probaron múltiples combinaciones de hiperparámetros para el modelo de RandomForestClassifier (n_estimators y max_depth). Para cada combinación de hiperparámetros, se evaluaron los modelos entrenados con datos sobremuestreados y submuestreados usando el conjunto de validación.

Se seleccionó el mejor modelo basado en el valor más alto de F1 Score obtenido en el conjunto de validación.

Hiperparámetros del mejor modelo:

n_estimators: 100
max_depth: 18

# **Prueba final**


In [ ]:
# Evaluación del mejor modelo
predictions_test = best_model.predict(features_test)
f1_score_best_model = f1_score(target_test, predictions_test)
auc_roc_best_model = roc_auc_score(target_test, best_model.predict_proba(features_test)[:, 1])

print("Mejor modelo con corrección de desequilibrio de clases:")
print("F1 Score:", f1_score_best_model)
print("AUC-ROC Score:", auc_roc_best_model)

Mejor modelo con corrección de desequilibrio de clases:
F1 Score: 0.6025
AUC-ROC Score: 0.8522892904413024


El mejor modelo fue evaluado en el conjunto de prueba.

Métricas del mejor modelo:

F1 Score: 0.6025 (valor mayor del 0.59 minimo requerido para aprobar la revisión)
AUC-ROC Score: 0.8523 (indicando una buena capacidad de discriminación entre las clases)
El enfoque de corregir el desequilibrio de clases mediante sobremuestreo resultó en un modelo de RandomForestClassifier con un F1 Score de 0.6025 y un AUC-ROC Score de 0.8523 en el conjunto de prueba.

Estos resultados indican que el modelo tiene una buena capacidad tanto en términos de precisión y recall (F1 Score) como en la discriminación entre clases (AUC-ROC Score).

# **Conclusión general**

El objetivo del proyecto era predecir si un cliente dejará el banco Beta Bank en el futuro cercano, utilizando datos históricos sobre el comportamiento de los clientes y la terminación de contratos. Dada la importancia de retener clientes existentes, se buscaba desarrollar un modelo con un valor F1 de al menos 0.59 para evaluar la capacidad del modelo en identificar correctamente los clientes que podrían abandonar el banco.

El modelo final no solo alcanzó sino que superó el umbral de F1 Score requerido, demostrando una buena capacidad para identificar correctamente a los clientes que podrían abandonar el banco.

El alto valor de AUC-ROC indica que el modelo tiene una fuerte capacidad de discriminación entre los clientes que abandonan y los que no.

Es posible predecir con buena precisión si un cliente dejará Beta Bank utilizando modelos de aprendizaje automático. Con un F1 Score de 0.6025 y un AUC-ROC de 0.8523, el modelo final se muestra como una manera de ayudar al banco a retener a sus clientes y tomar medidas proactivas para reducir la tasa de abandono.